In [2]:
import tensorflow as tf
from tensorflow.contrib import rnn

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data', one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [15]:
tf.reset_default_graph()

learning_rate = 0.01
num_steps = 10000
batch_size = 128
steps_print = 200

num_input = 28
timesteps = 28
num_hidden = 128
num_classes = 10

X = tf.placeholder(tf.float32, [None, timesteps, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])

In [16]:
def BiRNN(x, weights, biases):
    
    x = tf.unstack(x, timesteps, 1)
    
    lstm_fw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    lstm_bw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw, lstm_bw, x, dtype=tf.float32)
    
    return tf.matmul(outputs[-1], weights['out']) + biases['out'] 

In [17]:
weights = {
    'out': tf.Variable(tf.random_normal([2 * num_hidden, num_classes]))
}

biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [18]:

logits = BiRNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(prediction,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [22]:
with tf.Session() as sess:
    
    sess.run(init)
    
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape([-1, timesteps, num_input])
        
        sess.run([train_op], feed_dict = {X:batch_x, Y:batch_y})
        
        if step % steps_print == 0:
            loss, accu = sess.run([loss_op, accuracy], feed_dict = {X:batch_x, Y:batch_y})
            print("step", step, "minibatch loss", loss, "minibatch accuracy", accu)
            
    print("training finished!")
    
    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

step 200 minibatch loss 0.15712988 minibatch accuracy 0.9296875
step 400 minibatch loss 0.055097338 minibatch accuracy 0.984375
step 600 minibatch loss 0.12162349 minibatch accuracy 0.9765625
step 800 minibatch loss 0.01288471 minibatch accuracy 1.0
step 1000 minibatch loss 0.060132697 minibatch accuracy 0.984375
step 1200 minibatch loss 0.02557467 minibatch accuracy 0.9921875
step 1400 minibatch loss 0.050713725 minibatch accuracy 0.9921875
step 1600 minibatch loss 0.020397581 minibatch accuracy 0.9921875
step 1800 minibatch loss 0.014675704 minibatch accuracy 1.0
step 2000 minibatch loss 0.016424345 minibatch accuracy 1.0
step 2200 minibatch loss 0.026160851 minibatch accuracy 0.9921875
step 2400 minibatch loss 0.0041245036 minibatch accuracy 1.0
step 2600 minibatch loss 0.011448875 minibatch accuracy 1.0
step 2800 minibatch loss 0.036928743 minibatch accuracy 0.9921875
step 3000 minibatch loss 0.011150962 minibatch accuracy 1.0
step 3200 minibatch loss 0.008506944 minibatch accuracy